In [41]:
import csv
import sqlite3
import pandas as pd

from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

In [10]:
def create_table(conn, create_table_sql, drop_table_name=None):
    
    if drop_table_name: # You can optionally pass drop_table_name to drop the table. 
        try:
            c = conn.cursor()
            c.execute("""DROP TABLE IF EXISTS %s""" % (drop_table_name))
        except Error as e:
            print(e)
    
    try:
        c = conn.cursor()
        c.execute(create_table_sql)
    except Error as e:
        print(e)

Tables to make:

```
[Patients]
    [PatientID] INTEGER NOT NULL PRIMARY KEY
    [Age] REAL NOT NULL
    [Gender] TEXT NOT NULL
    [Married] TEXT NOT NULL
    [WorkType] TEXT NOT NULL
    [ResidenceType] TEXT NOT NULL

[HealthDetails]
    [PatientID] INTEGER NOT NULL PRIMARY KEY FOREIGN KEY TO Patients(PatientID)
    [HeartDisease] INTEGER NOT NULL
    [HyperTension] INTEGER NOT NULL
    [AvgGlucoseLevel] REAL NOT NULL
    [BMI] REAL
    [Smoker] TEXT NOT NULL

[Strokes]
    [PatientID] INTEGER NOT NULL PRIMARY KEY FOREIGN KEY TO Patients(PatientID)
    [Stroke] INTEGER NOT NULL
```

In [11]:
def create_strokes_db(datafile, db_name, delete_db = False):
    
    # Deletes existing database file, if desired
    if delete_db:
        import os
        if os.path.isfile(db_name):
            os.remove(db_name)
        else:
            raise FileNotFoundError(f"{db_name} does not exist.")
    
    # Create connection to database
    conn = sqlite3.connect(db_name)
    
    # Read and parse data
    header = None
    data = []
    with open(datafile) as csv_file:
        csv_reader = csv.reader(csv_file, delimiter=',')
        for row in csv_reader:
            if header == None:
                header = row
                continue
            data.append(row)

    # Data to insert
    patient_list = [(int(i[0]), i[2], i[1], i[5], i[6], i[7]) for i in data]
    health_dets = [(int(i[0]), i[4], i[3], i[-4], i[-3], i[-2]) for i in data]
    strokes = [(int(i[0]), int(i[-1])) for i in data]
    
    # Create tables
    create_patients_table_sql = """
    CREATE TABLE IF NOT EXISTS [Patients](
        [PatientID] INTEGER NOT NULL PRIMARY KEY,
        [Age] REAL,
        [Gender] TEXT,
        [Married] TEXTL,
        [WorkType] TEXT,
        [ResidenceType] TEXT);"""

    create_healthdetails_table_sql = """
    CREATE TABLE IF NOT EXISTS [HealthDetails](
        [PatientID] INTEGER NOT NULL PRIMARY KEY,
        [HeartDisease] INTEGER,
        [HyperTension] INTEGER,
        [AvgGlucoseLevel] REAL,
        [BMI] REAL,
        [Smoker] TEXT,
        FOREIGN KEY(PatientID) REFERENCES Patients(PatientID));
    """

    create_strokes_table_sql = """
    CREATE TABLE IF NOT EXISTS [Strokes](
        [PatientID] INTEGER NOT NULL PRIMARY KEY,
        [Stroke] INTEGER NOT NULL,
        FOREIGN KEY(PatientID) References Patients(PatientID));
    """
    
    # Insert statements
    insert_patients = """
    INSERT INTO Patients(
        PatientID,
        Age,
        Gender,
        Married,
        WorkType,
        ResidenceType) VALUES (?, ?, ?, ?, ?, ?)"""
    
    insert_healthdetails = """
    INSERT INTO HealthDetails(
        PatientID,
        HeartDisease,
        HyperTension,
        AvgGlucoseLevel,
        BMI,
        Smoker) VALUES (?, ?, ?, ?, ?, ?)"""

    insert_strokes = "INSERT INTO Strokes(PatientID, Stroke) VALUES (?, ?)"
    
    # Create tables and insert values
    with conn:
        cur = conn.cursor()
        create_table(conn, create_patients_table_sql, drop_table_name = "Patients")
        create_table(conn, create_healthdetails_table_sql, drop_table_name = "HealthDetails")
        create_table(conn, create_strokes_table_sql, drop_table_name = "Strokes")

        cur.executemany(insert_patients, patient_list)
        cur.executemany(insert_healthdetails, health_dets)
        cur.executemany(insert_strokes, strokes)
    
    conn.close()

In [12]:
create_strokes_db("stroke_data.csv", "stroke.db", delete_db = False)

In [14]:
conn = sqlite3.connect("stroke.db")

In [15]:
df = pd.read_sql_query("SELECT * FROM HealthDetails", conn)
df

,PatientID,HeartDisease,HyperTension,AvgGlucoseLevel,BMI,Smoker
0,67,0,0,92.97,N/A,formerly smoked
1,77,0,0,85.81,18.6,Unknown
2,84,0,0,89.17,31.5,never smoked
3,91,0,0,98.53,18.5,never smoked
4,99,0,0,108.89,52.3,Unknown
...,...,...,...,...,...,...
5105,72911,0,1,129.54,60.9,smokes
5106,72914,0,0,90.57,24.2,Unknown
5107,72915,0,0,172.33,45.3,formerly smoked
5108,72918,0,1,62.55,30.3,Unknown


In [204]:
# Join data into one dataframe
join_statement = """
SELECT
    p.PatientID, Age, Gender,
    ResidenceType, HeartDisease, HyperTension,
    AvgGlucoseLevel, BMI, Smoker,
    Stroke
FROM Patients AS p
INNER JOIN HealthDetails AS hd
ON p.PatientID = hd.PatientID
INNER JOIN Strokes AS s
ON s.PatientID = p.PatientID
"""

data = pd.read_sql_query(join_statement, conn)

In [205]:
data.head()

,PatientID,Age,Gender,ResidenceType,HeartDisease,HyperTension,AvgGlucoseLevel,BMI,Smoker,Stroke
0,67,17.0,Female,Urban,0,0,92.97,N/A,formerly smoked,0
1,77,13.0,Female,Rural,0,0,85.81,18.6,Unknown,0
2,84,55.0,Male,Urban,0,0,89.17,31.5,never smoked,0
3,91,42.0,Female,Urban,0,0,98.53,18.5,never smoked,0
4,99,31.0,Female,Urban,0,0,108.89,52.3,Unknown,0


In [206]:
import numpy as np
# Check for missing values
data["BMI"] = data["BMI"].apply(lambda x: np.nan if x == "N/A" else x) # Convert "N/A" to NaN

bmi_female, bmi_male, bmi_other = data[["Gender", "BMI"]].groupby("Gender").mean()["BMI"] # Gender-specific averages
bmi_female, bmi_male, bmi_other

(29.065757680358992, 28.64793635007459, 22.4)

In [207]:
data["Gender"].unique()
data["Smoker"].unique()

array(['Female', 'Male', 'Other'], dtype=object)

array(['formerly smoked', 'Unknown', 'never smoked', 'smokes'],
      dtype=object)

In [208]:
data.head()

,PatientID,Age,Gender,ResidenceType,HeartDisease,HyperTension,AvgGlucoseLevel,BMI,Smoker,Stroke
0,67,17.0,Female,Urban,0,0,92.97,NaN,formerly smoked,0
1,77,13.0,Female,Rural,0,0,85.81,18.6,Unknown,0
2,84,55.0,Male,Urban,0,0,89.17,31.5,never smoked,0
3,91,42.0,Female,Urban,0,0,98.53,18.5,never smoked,0
4,99,31.0,Female,Urban,0,0,108.89,52.3,Unknown,0


In [209]:
# Replace BMI missing values with gender-specific BMI averages
data["BMI"] = np.where((np.isnan(data["BMI"])) & (data["Gender"] == "Female"), bmi_female, data["BMI"])
data["BMI"] = np.where((np.isnan(data["BMI"])) & (data["Gender"] == "Male"), bmi_male, data["BMI"])

In [210]:
data.isna().sum()

PatientID          0
Age                0
Gender             0
ResidenceType      0
HeartDisease       0
HyperTension       0
AvgGlucoseLevel    0
BMI                0
Smoker             0
Stroke             0
dtype: int64

In [211]:
data.loc[data["Gender"] == "Other"]

,PatientID,Age,Gender,ResidenceType,HeartDisease,HyperTension,AvgGlucoseLevel,BMI,Smoker,Stroke
3926,56156,26.0,Other,Rural,0,0,143.33,22.4,formerly smoked,0


In [212]:
# Drop from dataset and reset indices
data.drop(index = 3926, inplace=True)
data.reset_index(drop=True, inplace=True)

In [214]:
data

,PatientID,Age,Gender,ResidenceType,HeartDisease,HyperTension,AvgGlucoseLevel,BMI,Smoker,Stroke
0,67,17.0,Female,Urban,0,0,92.97,29.065758,formerly smoked,0
1,77,13.0,Female,Rural,0,0,85.81,18.600000,Unknown,0
2,84,55.0,Male,Urban,0,0,89.17,31.500000,never smoked,0
3,91,42.0,Female,Urban,0,0,98.53,18.500000,never smoked,0
4,99,31.0,Female,Urban,0,0,108.89,52.300000,Unknown,0
...,...,...,...,...,...,...,...,...,...,...
5104,72911,57.0,Female,Rural,0,1,129.54,60.900000,smokes,0
5105,72914,19.0,Female,Urban,0,0,90.57,24.200000,Unknown,0
5106,72915,45.0,Female,Urban,0,0,172.33,45.300000,formerly smoked,0
5107,72918,53.0,Female,Urban,0,1,62.55,30.300000,Unknown,1


In [215]:
data.to_csv("processed_data.csv", index = False)